<img src="https://whatcar.vn/media/2018/09/car-lot-940x470.jpg"/>

# Прогнозирование стоимости автомобиля по характеристикам

## Цель данного учебного Data Science проекта
 * посторить модель машинного обучения на реальных данных (при необходимости и возможности, собрав их)
 * достичь наилучших показателей по соответствующим метрикам качества модели

## Достигнутые результаты

**Лучшее значение метрики MAPE:**
> 3.58%  
> Этот результат был достигнут в **[ДРУГОМ](https://www.kaggle.com/andreikukunov/car-price-prediction-auto-ru-2021-09) ноутбуке**  
> на **спаршеном датасете** от сентября 2009 года


**Лучшее значение Kaggle Score:**
> 17.78%  
> Этот результат был достигнут в **ДАННОМ ноутбуке**  
> на **базовом датасете**


## Что сделано

### Предобработка

#### [С базовым тренировочным датасетом](https://www.kaggle.com/andreikukunov/ak-from-local-sf-dst-car-price-prediction-v16)

> **Отбор колонок**
> * исключены коррелирующие между собой признаки
> * оставлены важные для модели м.о. признаки


> **Очистка и коррекция данных**
> * исправлена неоднородность данных между обучающей и тестовой выборками
> * пустоты в данных заполнены или удалены  


> **Обогащение данных**
> * создан работающий код для парсинга региона объявления по ссылке


### Подобраны модели МО:
> * CatBoostRegressor -- подобраны лучшие параметры и сделана логарифмизация таргета
> * XGBoostRegressor -- получен схожий результат с CatBoost
> * StackingRegressor -- обучена комбинация CatBoost и XGBoost

# Содержание:

- [Setup&Load](#Setup&Load)
- [EDA](#EDA)
  * [Первый взгляд на данные](#Первый-взгляд-на-данные)
  * [Data Preprocessing](#Data-Preprocessing)
    + [Отбираем колонки](#Отбираем-колонки)
    + [Немного аналитики](#Немного-аналитики)
    + [Сравниваем Test и Train по моделям автомобилей](#Сравниваем-Test-и-Train-по-моделям-автомобилей)
    + [Обрабатываем](#Обрабатываем)
    + [Общая подготовка датасетов](#Общая-подготовка-датасетов)
- [Перебираем модели](#Перебираем-модели)
    + [Train Split](#Train-Split)
  * [Model 1 : Создадим "наивную" модель](#Model-1-:-Создадим-"наивную"-модель)
  * [Model 2 : CatBoost](#Model-2-:-CatBoost)
    + [CatBoostRegressor базовый](#CatBoostRegressor-базовый)
    + [CatBoostRegressor: подбор параметров](#CatBoostRegressor:-подбор-параметров)
    + [Log Traget](#Log-Traget)
  * [Model 3 : XGBoostRegressor -- пробуем](#Model-3-:-XGBoostRegressor----пробуем)
  * [Model 4 : StackingRegressor](#Model-4-:-StackingRegressor)
- [Submission](#ubmission)
- [Приложение -- pandas_profiling](#Приложение----pandas_profiling)

# Setup&Load

In [1]:
#%pip install catboost
#%pip install ipywidgets
#%pip install tqdm
#%pip install pandas-profiling[notebook]
#%pip install CurrencyConverter hyperopt xgboost
#%pip install xgboost

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import sys
from sklearn.model_selection import train_test_split
from sklearn.ensemble import StackingRegressor
#from sklearn.model_selection import KFold
#from tqdm.notebook import tqdm
from catboost import CatBoostRegressor
import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score   #, make_scorer
from sklearn.linear_model import LinearRegression
#from sklearn.preprocessing import LabelEncoder

In [3]:
pd.set_option('display.max_columns', None) # чтобы выводить все столбцы
pd.options.display.max_colwidth = 100   # задать кол-во символов для показа

In [4]:
#!jupyter nbextension enable --py --sys-prefix widgetsnbextension


In [5]:
from pprint import pprint  

In [6]:
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)

In [7]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

In [8]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [9]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))

In [10]:
VERSION    = 16
DIR_TRAIN  = '../input/parsing-all-moscow-auto-ru-09-09-2020/' # подключил к ноутбуку внешний датасет
DIR_TEST   = '../input/sf-dst-car-price-prediction/'
VAL_SIZE   = 0.20   # 20%

In [11]:
#VERSION    = 16
#DIR_TRAIN  = '' # подключил к ноутбуку внешний датасет
#DIR_TEST   = ''
#VAL_SIZE   = 0.20   # 20%

In [12]:
!ls '../input'

In [13]:
#train = pd.read_csv('all_auto_ru_09_09_2020.zip') # датасет для обучения модели
#test = pd.read_csv('test.zip')
#sample_submission = pd.read_csv('sample_submission.csv')

In [14]:
train = pd.read_csv(DIR_TRAIN+'all_auto_ru_09_09_2020.csv') # датасет для обучения модели
test = pd.read_csv(DIR_TEST+'test.csv')
sample_submission = pd.read_csv(DIR_TEST+'sample_submission.csv')

# EDA

## Первый взгляд на данные

In [15]:
train.sample(1)

In [16]:
train.info()

In [17]:
test.sample(1)

In [18]:
test.info()

In [19]:
# Список марок в test
brands = [brand for brand in test.brand.unique()]    # .lower()
brands

In [20]:
# Оставляем в train только те марки автомобилей, которые есть в test
train = train[train.brand.isin(test.brand.unique())]
train.sample(1)

In [21]:
# Сколько европеских и японских авто представлено в выборке.
# Но категоризация по vendor вряд ли нужна в модели... 
test.groupby(['vendor', 'brand']).name.count()#.index

## Data Preprocessing

### Отбираем колонки

In [22]:
test.columns, train.columns

In [23]:
# Смотрим на корреляции признаков, чтобы выбрать поелзные для модели
plt.figure(figsize=(15, 8));
sns.heatmap(train.corr(), vmin=-1, vmax=1, annot = True, cmap = 'coolwarm')

In [24]:
def columns_inclusion(DF1, DF2, DF3):
    '''
    Функция для сравнения датафреймов по присутствующим в них колонкам
    '''
    col_inc = pd.DataFrame()
    col_inc['all_cols'] = list(set(DF1.columns).union(DF2.columns, DF3.columns))
    col_inc['isin_DF1'] = col_inc.all_cols.isin(DF1.columns)
    col_inc['isin_DF2'] = col_inc.all_cols.isin(DF2.columns)
    col_inc['isin_DF3'] = col_inc.all_cols.isin(DF3.columns)

    return col_inc.sort_values('isin_DF3')

In [26]:
# Колонки test, отсутствующие в train
# И наоборот
set(test.columns).difference(train.columns), set(train.columns).difference(test.columns)


In [27]:
# Общие колонки
set(test.columns).intersection(train.columns)


In [28]:
# Общие колонки, но еще не включенные в data
#set(test.columns).intersection(train.columns).difference(data)

In [29]:
# Привод -- единообразить
test.Привод.value_counts(), train.Привод.value_counts(),

In [30]:
# Владельцы -- единообразить
# подвох в строках
#test.Владельцы.value_counts(), train.Владельцы.value_counts(),
test.Владельцы.values, train.Владельцы.values

In [31]:
# трансмиссия -- единообразить
test.vehicleTransmission.value_counts(), train.vehicleTransmission.value_counts(),

In [32]:
# модель -- переименовать колонку, оставить только имеющиеся в тестовой выборке
test.model_name.value_counts(), train.model.value_counts(),

In [33]:
# топливо ОК
test.fuelType.value_counts(), train.fuelType.value_counts(), 

In [34]:
# лошадиные силы очистить
test.enginePower.value_counts(), train.enginePower.value_counts(), 

In [35]:
# цвета единообразить 
test.color.value_counts(), train.color.value_counts(), 

In [36]:
# Состояние -- не информативно и пусто
test.Состояние.value_counts(), train.Состояние.value_counts(), 

### Немного аналитики

In [39]:
# Машины 2020 года выпуска как много их
train.productionDate.hist(bins=100)

In [40]:
# Автомобили без пробега, год производства. Поробуем их не брать ...
train[train.mileage<100].productionDate.value_counts()

# --> их удаление привело к ухудшению метрики и score

In [41]:
# Дата парсинга тестовой выборки -- октябрь 20-го, 
# что всего на месяц отличается от приложенной к бейзлайну демонстрационной выборке для обучения.
# Поэтому не корректируем цену автомобилей для этой выборки.
 
pd.Timestamp(test.parsing_unixtime.mean(), unit='s')

Согласно статистике от [Autostat]('https://www.autostat.ru/infographics/48987/'), цены на авторынке не значительно выросли за период между парсингом данных тестовой и тренировочной выборок.  


![autostat](https://www.autostat.ru/application/includes/blocks/big_photo/images/cache/000/094/996/aba1c0d9-670-0.jpg)  


### Сравниваем Test и Train по моделям автомобилей

In [42]:
# Колонка с названиями моделей автомобилей заполнена довольно "грязно".
# Попробуем почистить ее и оставить, т.к. она является довольно важным признаком для предсказаний.

# Обрезаем колонку с моделями по первому "слову" в ячейке, очищаем от окружающих пробелов

# Переименовываем название признаков
train.rename(columns={'model': 'model_name'}, inplace=True)

test.model_name = test.model_name.apply(lambda x: x.lower().split()[0].strip() if isinstance(x, str) else x)
train.model_name = train.model_name.apply(lambda x: x.lower().split()[0].strip() if isinstance(x, str) else x)

In [43]:
# Смотрим, как отличаются выборки по количеству моделей
test.model_name.value_counts(normalize=True)[:20], train.model_name.value_counts(normalize=True)[:20]

# --> Отличия небольшие, но попробуем уравновесить их 

In [44]:
# Создадим словарь с весами по каждой "модели", чтобы затем попробовать скорректировать дисбаланс в выборках

# Соотношение количества моделей автомобилей в выборках, оставляем только совпадающие модели

models_rate = test.model_name.value_counts(normalize=True)[:].combine(
    train.model_name.value_counts(normalize=True)[:], 
    np.divide).sort_values(ascending=False).fillna(1).to_dict() 

#models_rate

In [45]:
# Проверяем, все ли 'модели' вошли в словарь
len(models_rate), len(train.model_name.unique())+len(test.model_name.unique())

### Обрабатываем

In [46]:
# Словарь перекодировки цветов 🌻
hex_to_rus = {'040001':'чёрный', 
 'FAFBFB':'белый', 
 '97948F':'серый', 
 'CACECB':'серебристый', 
 '0000CC':'синий', 
 '200204':'коричневый', 
 'EE1D19':'красный', 
 '007F00':'зелёный', 
 'C49648':'бежевый', 
 '22A0F8':'голубой', 
 '660099':'пурпурный', 
 'DEA522':'золотистый', 
 '4A2197':'фиолетовый', 
 'FFD600':'жёлтый', 
 'FF8649':'оранжевый', 
 'FFC0CB':'розовый', 
}

In [47]:
# Переименовываем название признаков  -- сделано выше
#train.rename(columns={'model': 'model_name'}, inplace=True)

# Удаляем лишнее из enginePower в тестовой выборке и превращаем в числовой формат
test.enginePower = test.enginePower.apply(lambda x: float(x[:-4]))

# Заменяем значения в vehicleTransmission на соответствующие из другой выборки
test.vehicleTransmission = test.vehicleTransmission.replace(to_replace={'автоматическая':'AUTOMATIC', 'механическая':'MECHANICAL', 'роботизированная':'ROBOT', 'вариатор':'VARIATOR'})

# Заменяем значения в Владельцы на соответствующие из другой выборки
test.Владельцы = test.Владельцы.replace(to_replace={'1\xa0владелец':1.0, '2\xa0владельца':2.0, '3 или более':3.0})

# Заменяем значения в color на соответствующие из другой выборки
train.color = train.color.replace(to_replace = hex_to_rus)




In [48]:
# Отсеиваем автомобили без пробега

#train.mileage = train.mileage!=0

# Но это ухудшает метрику

In [49]:
# Интересный способ воссоздания признака (зависимого). Но -- ухудшает метрику.

#vendor_dict = {k:v for v,k in test.groupby(['vendor', 'brand']).name.count().index}
#print(vendor_dict)
#train['vendor'] = train.brand.map(vendor_dict)
#train.vendor.unique()

In [50]:
# Пробуем скорректировать дисбаланс моделей в выборках 
# Добавляем в каждую выборку колонку с информацией о соотношении моделей 

test['models_rate'] = test.model_name.replace(to_replace = models_rate)#.apply(lambda x: x.replace(0) if isinstance(x, str) else x)    #.isna().value_counts()
train['models_rate'] = 1 # train.model_name.replace(to_replace = models_rate)#.apply(lambda x: x.replace(0) if isinstance(x, str) else x)    #.isna().value_counts()

In [51]:
# Корректировка цены по инфляции на рынке автомобилей -- не производим!
#train.price = train.price*0.7

In [52]:
# колонки в итоге

#columns = ['bodyType', 'brand', 'productionDate', 'mileage', 'model_name', 'enginePower', 'vehicleTransmission', 'Владельцы', 'Привод', 'fuelType']
#columns = ['bodyType', 'brand', 'productionDate', 'mileage', 'model_name', 'enginePower', 'Привод', 'fuelType']
#columns = ['productionDate', 'mileage', 'model_name', 'enginePower', 'vehicleTransmission', 'Владельцы', 'fuelType', 'color']

columns = ['bodyType', 'brand', 'productionDate', 'mileage', 'model_name', 'enginePower', 'vehicleTransmission', 'Владельцы', 'Привод', 'fuelType', 'color', 'models_rate']    # , 'models_rate' , 'vendor'



#### Пустоты 

In [53]:
train[columns].isna().sum()

In [54]:
# Работаем с пропусками

train.dropna(subset=['productionDate','mileage', 'Привод', 'vehicleTransmission', 'enginePower', 'bodyType',  ], inplace=True)
train.Владельцы.fillna(3.0, inplace=True) # train.Владельцы.mode()

train.dropna(subset=['price'], inplace=True)

train[columns].isna().sum()

### Общая подготовка датасетов

In [56]:
# Оставляем в выборках только отобранные колонки

df_train = train[columns]
df_test = test[columns]

In [ ]:
y = train['price']

In [ ]:
# ВАЖНО! для корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест (или submission!)

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

In [ ]:
data

In [ ]:
# Помечаем категориальные признаки (для CatBoost) и кодируем их

#columns.drop(['mileage', 'enginePower', 'Владельцы'])
#cat_columns = ['bodyType', 'brand', 'productionDate', 'model_name', 'vehicleTransmission', 'Привод', 'fuelType']
#cat_columns = ['bodyType', 'brand', 'productionDate', 'model_name', 'Привод', 'fuelType']
#cat_columns = ['productionDate', 'model_name', 'vehicleTransmission', 'fuelType', 'color']

cat_columns = ['bodyType', 'brand', 'productionDate', 'model_name', 'vehicleTransmission', 'Привод', 'fuelType', 'color']   #, 'vendor'


#for colum in ['bodyType', 'brand', 'engineDisplacement']:
for colum in cat_columns:
    data[colum] = data[colum].astype('category').cat.codes

In [ ]:
data

# Перебираем модели

### Train Split

In [ ]:
X = data.query('sample == 1').drop(['sample'], axis=1)
X_sub = data.query('sample == 0').drop(['sample'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)



## Model 1 : Создадим "наивную" модель 
Эта модель будет предсказывать среднюю цену по модели двигателя (engineDisplacement). 
C ней будем сравнивать другие модели.




In [ ]:
tmp_train = X_train.copy()
tmp_train['price'] = y_train

In [ ]:
# Находим median по экземплярам engineDisplacement в трейне и размечаем тест
#predict = X_test['engineDisplacement'].map(tmp_train.groupby('engineDisplacement')['price'].median())
predict = X_test['enginePower'].map(tmp_train.groupby('enginePower')['price'].median())

#оцениваем точность
print(f"Точность наивной модели по метрике MAPE: {(mape(y_test, predict.values))*100:0.2f}%")

## Model 2 : CatBoost
![](https://pbs.twimg.com/media/DP-jUCyXcAArRTo.png:large)   


У нас в данных практически все признаки категориальные. Специально для работы с такими данными была создана очень удобная библиотека CatBoost от Яндекса. [https://catboost.ai](http://)     
На данный момент **CatBoost является одной из лучших библиотек для табличных данных!**

#### Полезные видео о CatBoost (на русском):
* [Доклад про CatBoost](https://youtu.be/9ZrfErvm97M)
* [Свежий Туториал от команды CatBoost (практическая часть)](https://youtu.be/wQt4kgAOgV0) 

### CatBoostRegressor базовый

In [ ]:
#model = CatBoostRegressor(iterations = 5000,
#                          random_seed = RANDOM_SEED,
#                          eval_metric='MAPE',
#                          custom_metric=['R2', 'MAE'],
#                          silent=True,
#                          use_best_model = True
#                         )
#model.fit(X_train, y_train,
#         #cat_features=cat_features_ids,
#         #cat_features=cat_columns,
#         eval_set=(X_test, y_test),
#         verbose_eval=0,
#         use_best_model=True,
#         plot=True
#         )

#model.save_model('catboost_single_model_baseline.model')

In [ ]:
# оцениваем точность
# base MAPE: 18.11%

#predict = model.predict(X_test)
#print(f"Точность модели по метрике MAPE: {(mape(y_test, predict))*100:0.2f}%")

# --> MAPE: 16.09% с базовой обработкой
# --> MAPE: 15.37% с отсеиванием моделей авто
# --> MAPE: 15.51% без малозначимых признаков
# --> MAPE: 15.35% с цветом
# --> MAPE: 15.35% с понижением цены на 30%
# --> MAPE: 16.76% с удалением коррелирующих переменных
# --> MAPE: 16.08% с удалением автомобилей без пробега
# --> MAPE: 14.25% на 5000 итерациях (было по умолчанию) 

#### Оцениваем важность признаков

In [ ]:
# Важность признаков
#pd.Series(model.get_feature_importance(), index = X_train.columns).plot(kind='bar')   #.sort_values(ascending=False)

In [ ]:
# В удобной таблице
#model.get_feature_importance(prettified=True)

In [ ]:
# Общие колонки test и train, но еще не квлюченные в data
set(test.columns).intersection(train.columns).difference(data)

In [ ]:
# Колонки включенные и не включенные
columns_inclusion(test, train, data)


In [ ]:
# Смотрим на все использованные параметры
#model.get_all_params()

### CatBoostRegressor: подбор параметров
которые особо влияют на качество модели

In [ ]:

#model = CatBoostRegressor(iterations = 6000, random_seed = RANDOM_SEED, eval_metric='MAPE', silent=True)

#grid = {'learning_rate': [0.03, 0.1], 'depth': [4, 6, 10], 'l2_leaf_reg': [1, 3, 5, 7, 9]}

#randomized_search_result = model.randomized_search(grid, X_train, y_train, plot=True)

# --> MAPE: 14.08% с подбором ()

In [ ]:
# Выбранные параметры. 
# Оставим только 6000 итераций. Лучшая из них -- примерно на 

#cbr_best_params = model.get_params()
cbr_best_params = {'iterations': 6500,
                'loss_function': 'RMSE',
                'random_seed': 42,
                'silent': True,
                'eval_metric': 'MAPE',
                'depth': 10,
                'l2_leaf_reg': 9,
                'learning_rate': 0.03}

#### Важность признаков

In [ ]:
# Для запуска ячеек выше

### Log Traget
Попробуем взять таргет в логорифм - это позволит уменьшить влияние выбросов на обучение модели (используем для этого np.log и np.exp).    
(В принциепе мы можем использовать любое приобразование на целевую переменную. Например деление на курс доллара, евро или гречки :) в дату сбора данных, смотрим дату парсинга в тесте в **parsing_unixtime**()


**Применим подобранные параметры.**

In [ ]:
model = CatBoostRegressor(**cbr_best_params)

model.fit(X_train, np.log(y_train), eval_set=(X_test, np.log(y_test)), verbose_eval=0, use_best_model=True, plot=True)

model.save_model('catboost_single_model_2_baseline.model')

In [ ]:
# Разлогарифмируем предсказания модели
predict_test = np.exp(model.predict(X_test)) 

print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")

# --> MAPE: 12.20% при 14.25% без логарифмизации
# --> MAPE: 12.09% при 14.08% без логарифмизации -- всё с подбором параметров

In [ ]:
# submission для CatBoost с логарифмированием
predict_submission = np.exp(model.predict(X_sub)) #-- не применяем

## Model 3 : XGBoostRegressor -- пробуем
![](https://cdn3.vectorstock.com/i/1000x1000/74/02/coffee-boost-energy-vector-19917402.jpg)

In [ ]:

#xgb_log = xgb.XGBRegressor(
#    objective='reg:squarederror', 
#    colsample_bytree=0.5,               
#    learning_rate=0.05, 
#    max_depth=12, 
#    alpha=1,                   
#    n_estimators=1000,
#    random_state=RANDOM_SEED,
#    verbose=1, 
#    n_jobs=-1,
#)


#xgb_log.fit(X_train, np.log(y_train))   #!!!

## For MAPE metric (or any other), we need the predictions of the model
#y_pred = np.exp(xgb_log.predict(X_test))

#print('mean_squared_error:', mean_squared_error(y_test, y_pred))
#print('r2_score:', r2_score(y_test, y_pred))
#xgb_log_mape_value = mape(y_test, y_pred)
#xgb_log_mape_values.append(xgb_log_mape_value)
#print('log_mape:', xgb_log_mape_value)

(The MAPE metric in the notebook is: 14.78%, on the leaderboard: 10.75%. )

--> MAPE: 12.04%

In [ ]:
#xgb.plot_importance(xgb_log)
#plt.show()

Примечательно, что важность признаков для XGBoostRegressor несколько отличается от важности признаков для CatBoostRegressor
- descr_words_count
- mileage
- mileage_per_year
- enginePower_log2
- model_name

## Model 4 : StackingRegressor
![](https://www.treehugger.com/thmb/MPyNjODcEJqfxa9B_PO4r-B41D4=/887x665/smart/filters:no_upscale()/__opt__aboutcom__coeus__resources__content_migration__mnn__images__2018__08__Stacking-rocks-beach-cairns-ca8fcb6c081d40409bf960865c2315b5.jpg)

Here it's intesting to try to stack the models that have demostrated the best results: ExtraTreeRegressor and XGBRegressor with the LinearRegression meta model:

In this case the stacking of the model can also improve the metrics value and prevent the overfitting.

In [ ]:
#estimators = [
##    ('etr', ExtraTreesRegressor(random_state=RANDOM_SEED, n_jobs=-1, verbose=1)),
#    #('etr', CatBoostRegressor(random_state=RANDOM_SEED, iterations = 5000, eval_metric='MAPE', silent=True)),   
#    ('etr', CatBoostRegressor(**cbr_best_params)),   
#    ('xgb', xgb.XGBRegressor(objective='reg:squarederror', colsample_bytree=0.5, learning_rate=0.05, max_depth=12, alpha=1, n_jobs=-1, n_estimators=1000, random_state=RANDOM_SEED))
#]

#sr_log = StackingRegressor(
#    estimators=estimators,
#    final_estimator=LinearRegression()
#)

## For training, fit() is used
#sr_log.fit(X_train, np.log(y_train))

## For MAPE metric (or any other), we need the predictions of the model
#y_pred = np.exp(sr_log.predict(X_test))

#print(f"The MAPE mertic for the default StackingRegressor model: {(mape(y_test, y_pred) * 100):0.2f}%.")

(The MAPE metric in the notebook is: 15.39%, on the leaderboard: %10.80. )

--> MAPE: 11.94%  
--> MAPE: 11.91% с подобранными параметрами

--> MAPE: 11.91% с почищенными моделями

In [ ]:
#predict_test = np.exp(sr_log.predict(X_test))
#predict_submission = np.exp(sr_log.predict(X_sub))


# Submission

In [ ]:
sample_submission['price'] = predict_submission
#sample_submission.to_csv(f'submission_2_v{VERSION}.csv', index=False)
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)

В итоге получили **MAPE 27%** на ЛБ!

Большая разница в ошибке может указывать на то что тест и трейн имеют различия по выборке или то что данные в трейне могли уже устареть и их нужно обновлять.

# What's next?
Или что еще можно сделать, чтоб улучшить результат:

* Спарсить свежие данные 
* Посмотреть, что можно извлечь из признаков или как еще можно обработать признаки
* Сгенерировать новые признаки
* Попробовать подобрать параметры модели
* Попробовать другие алгоритмы и библиотеки ML
* Сделать Ансамбль моделей, Blending, Stacking

Подробный чек лист: https://docs.google.com/spreadsheets/d/1I_ErM3U0Cs7Rs1obyZbIEGtVn-H47pHNCi4xdDgUmXY/edit?usp=sharing

# Приложение -- pandas_profiling

In [ ]:
from pandas_profiling import ProfileReport

df = data
profile = ProfileReport(df)
profile.to_notebook_iframe()